<a href="https://colab.research.google.com/github/Crane-YU/machine_learning_keras/blob/master/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print("The shape of training example is: ", train_data.shape)
print("The shape of test_data example is: ", test_data.shape)

Using TensorFlow backend.


The shape of training example is:  (404, 13)
The shape of test_data example is:  (102, 13)


In [2]:
# normalizing data
mean = train_data.mean(axis=0)
print("The shape of the mean is:", mean.shape)

train_data -= mean
std = train_data.std(axis=0)
train_data /= std
print("The shape of train_data is: ", train_data.shape[1])

test_data -= mean
test_data /= std

The shape of the mean is: (13,)
The shape of train_data is:  13


In [0]:
from keras import models
from keras import layers

def build_model():
  # Because we will need to instantiate
  # the same model multiple time,
  # we use a function to construct it.
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu',
  input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  return model

In [9]:
import numpy as np

k = 4
num_val_samples = len(train_data)//k
num_epochs = 100
all_scores = []

for i in range(k):
  print('processing fold #', i)
  # Prepare the validation data: data from partition # k
  val_data = train_data[i*num_val_samples: (i+1)*num_val_samples]
  val_targets = train_targets[i*num_val_samples: (i+1)*num_val_samples]
  
  # Prepare the training data: data from all other partitions
  partial_train_data = np.concatenate([train_data[:i * num_val_samples], train_data[(i + 1)*num_val_samples:]], axis=0)
  partial_train_targets = np.concatenate([train_targets[:i * num_val_samples], train_targets[(i + 1) * num_val_samples:]], axis=0)

  # Build the Keras model (already compiled)
  model = build_model()

  # Train the model (in silent mode, verbose=0)
  model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)

  # Evaluate the model on the validation data
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [10]:
print("The element in all_scores is: ", all_scores)
print("The average value is: ", np.mean(all_scores))

The element in all_scores is:  [2.318483121324294, 2.966167143075773, 2.390640608154901, 2.377374534559722]
The average value is:  2.5131663517786724


In [12]:
num_epochs = 500
all_mae_history = []

for i in range(k):
  print("processing fold #", i)
  
  # Prepare the validation data: data from partition # k
  val_data = train_data[i*num_val_samples: (i+1)*num_val_samples]
  val_targets = train_targets[i*num_val_samples: (i+1)*num_val_samples]
  
  # Prepare the training data: data from all other partitions
  partial_train_data = np.concatenate([train_data[:i*num_val_samples], train_data[(i+1)*num_val_samples:]], axis=0)
  partial_train_targets = np.concatenate([train_targets[:i*num_val_samples], train_targets[(i+1)*num_val_samples:]], axis=0)
  
  # Build the Keras model (already compiled)
  model = build_model()
  
  # Save the values
  history = model.fit(partial_train_data, partial_train_targets, validation_data=(val_data, val_targets), epochs=num_epochs, batch_size=1, verbose=0)
  mae_history = history.history['val_mean_absolute_error']
  
  # append the value
  all_mae_history.append(mae_history)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [0]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()